In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os 
import glob
import shutil

In [3]:
path0 = '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem'

In [ ]:
#path0 = '/content/drive/MyDrive/Colab Notebooks/의료영상처리및실습/final_project/brats18_train' # 데이터 경로

In [ ]:
path_list = sorted(glob.glob(os.path.join(path0,'Brats18*')))

In [ ]:
len(path_list)

200

In [ ]:
path_list

['/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_11_1',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_11_1_pro',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_13_1',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_13_1_pro',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_17_1',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_17_1_pro',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_19_1',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_19_1_pro',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_21_1',
 '/content/drive/MyDrive/21_2_Image_process_project/brats18_train_problem/Brats18_2013_21_1_pro',
 '/content/drive/MyDrive/21_2_Image_proc

In [ ]:
data_ind = np.arange(0,100)
data_dicts = [
    {
        "image": os.path.join(path_list[idx],
                                    "t1ce.nii.gz"),
        "label": os.path.join(path_list[idx],
                                    "seg.nii.gz"),
    }
    for idx in data_ind
]

In [ ]:
print(data_dicts[1]['label'])

/content/drive/MyDrive/21_2_Image_process_project/brats18_train/Brats18_2013_13_1/seg.nii.gz


##image processing

In [ ]:
for i in range(100):
  # t1ce matrix 불러오기
  img = nib.load(data_dicts[i]['image'])
  im = img.get_fdata()
  im_max_contrast = np.max(im)
  im_min_contrast = np.min(im)
  # 불러온 데이터 histogram 만들기
  cont_hist,cont_bins = np.histogram(im.flatten(),int(im_max_contrast),[int(im_min_contrast),int(im_max_contrast)])
  cont_bins = list(map(int,cont_bins)) #[Python] 리스트 각 원소들의 타입 변경
  hist_data = [[h,b] for h,b in zip(cont_hist,cont_bins)]
  use = hist_data[1:] # 0은 값이 너무 많아서 제외
  a = max(use)
  b = a[1]
  th = 2*b # thresholding
  # thresholding값 적용하기
  im[im>th] = th
  processed_max_contrast = im.max()
  processed_min_contrast = im.min()

  #새로운 directory 생성
  new_path = path_list[i] + "_pro"
  if not os.path.exists(new_path):
    os.mkdir(new_path)
  #print(new_path)
  
  #새로 만든 processed_t1ce.nii.gz 저장하기
  nifti = nib.Nifti1Image(im, img.affine, img.header)
  nib.save(nifti,os.path.join(new_path,"t1ce.nii.gz"))
  #label 복사하기
  shutil.copy(data_dicts[i]['label'], new_path)
  

  # unit8 (0-255)
  # im_8 = 255*((im-im.min())/(im.max()-im.min()))
  # nifti = nib.Nifti1Image(im_8, img.affine, img.header)
  # nib.save(nifti,os.path.join(path0,'unit8_processed.nii.gz'))
  #